# Kernel Semántico con Integración del Servidor MCP de OpenBnB

Este cuaderno demuestra cómo usar Kernel Semántico con el servidor MCP real de OpenBnB para buscar alojamientos reales de Airbnb utilizando MCPStdioPlugin. Para el acceso a LLM, utiliza Azure AI Foundry. Para configurar tus variables de entorno, puedes seguir la [Lección de Configuración](/00-course-setup/README.md)


## Importar los paquetes necesarios


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Creando la Conexión del Plugin MCP

Nos conectaremos al [servidor MCP de OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizando MCPStdioPlugin. Este servidor ofrece funcionalidad de búsqueda de Airbnb a través del paquete @openbnb/mcp-server-airbnb.


## Creando el Cliente

En este ejemplo, utilizaremos Azure AI Foundry para nuestro acceso a LLM. Asegúrate de que tus variables de entorno estén configuradas correctamente.


## Configuración del Entorno

Configura los ajustes de Azure OpenAI. Asegúrate de tener las siguientes variables de entorno configuradas:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Entendiendo la Integración de OpenBnB MCP

Este cuaderno se conecta al **servidor real de OpenBnB MCP**, que proporciona funcionalidad de búsqueda real de Airbnb.

### Cómo funciona:

1. **MCPStdioPlugin**: Utiliza comunicación de entrada/salida estándar con el servidor MCP.
2. **Paquete NPM Real**: Descarga y ejecuta `@openbnb/mcp-server-airbnb` a través de npx.
3. **Datos en Vivo**: Devuelve datos reales de propiedades de Airbnb desde sus APIs.
4. **Descubrimiento de Funciones**: El agente descubre automáticamente las funciones disponibles del servidor MCP.

### Funciones Disponibles:

El servidor OpenBnB MCP típicamente ofrece:
- **search_listings** - Buscar propiedades de Airbnb por ubicación y criterios.
- **get_listing_details** - Obtener información detallada sobre propiedades específicas.
- **check_availability** - Verificar disponibilidad para fechas específicas.
- **get_reviews** - Recuperar reseñas de propiedades.
- **get_host_info** - Obtener información sobre los anfitriones de las propiedades.

### Requisitos Previos:

- **Node.js** instalado en tu sistema.
- **Conexión a Internet** para descargar el paquete del servidor MCP.
- **NPX** disponible (incluido con Node.js).

### Probando la Conexión:

Puedes probar el servidor MCP manualmente ejecutando:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Esto descargará e iniciará el servidor OpenBnB MCP, al cual Semantic Kernel se conectará para obtener datos reales de Airbnb.


## Ejecutar el Agente con el Servidor OpenBnB MCP

Ahora ejecutaremos el Agente de IA que se conecta al servidor OpenBnB MCP para buscar alojamientos reales de Airbnb en Estocolmo para 2 adultos y 1 niño. Siéntete libre de cambiar la lista `user_inputs` para modificar los criterios de búsqueda.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Resumen  
¡Felicidades! Has construido con éxito un agente de IA que se integra con la búsqueda de alojamientos en el mundo real utilizando el Protocolo de Contexto de Modelo (MCP):

Tecnologías utilizadas:  
Semantic Kernel - Para construir agentes inteligentes con Azure OpenAI  
Azure AI Foundry - Para capacidades de LLM y finalización de chat  
MCP (Model Context Protocol) - Para integración estandarizada de herramientas  
OpenBnB MCP Server - Para funcionalidad real de búsqueda en Airbnb  
Node.js/NPX - Para ejecutar el servidor MCP externo  

Lo que has aprendido:  
Integración MCP: Conectar agentes de Semantic Kernel a servidores MCP externos  
Acceso a datos en tiempo real: Buscar propiedades reales de Airbnb a través de APIs en vivo  
Comunicación por protocolo: Usar comunicación stdio entre el agente y el servidor MCP  
Descubrimiento de funciones: Descubrir automáticamente funciones disponibles de servidores MCP  
Respuestas en streaming: Capturar y registrar llamadas a funciones en tiempo real  
Renderizado HTML: Formatear respuestas del agente con tablas estilizadas y pantallas interactivas  

Próximos pasos:  
Integrar servidores MCP adicionales (clima, vuelos, restaurantes)  
Construir un sistema multi-agente combinando protocolos MCP y A2A  
Crear servidores MCP personalizados para tus propias fuentes de datos  
Implementar memoria de conversación persistente entre sesiones  
Desplegar el agente en Azure Functions con orquestación de servidores MCP  
Agregar autenticación de usuarios y capacidades de reserva  

Ventajas clave de la arquitectura MCP:  
Estandarización: Protocolo universal para conectar agentes de IA a herramientas externas  
Datos en tiempo real: Acceso a información actualizada y en vivo de diversos servicios  
Extensibilidad: Integración sencilla de nuevas fuentes de datos y herramientas  
Interoperabilidad: Funciona en diferentes marcos de IA y plataformas de agentes  
Separación de responsabilidades: Distinción clara entre la lógica de IA y el acceso a datos externos  



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
